In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
print(tf.__version__)

2.5.0


In [2]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

In [3]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

In [4]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

In [5]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

In [6]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.RandomNormal()

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(2):
            self.model.add(dense(256, weight_init))
            self.model.add(relu())

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):
        x = self.model(x)

        return x

In [7]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.RandomNormal()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(2) :
        model.add(dense(256, weight_init))
        model.add(relu())

    model.add(dense(label_dim, weight_init))

    return model

In [8]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

In [9]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_relu'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

In [10]:

if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):                
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)

                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 1.0582, train_loss: 2.16529417, train_accuracy: 0.3906, test_Accuracy: 0.2324
Epoch: [ 0] [    1/  468] time: 1.3464, train_loss: 2.13100982, train_accuracy: 0.3672, test_Accuracy: 0.3172
Epoch: [ 0] [    2/  468] time: 1.5808, train_loss: 2.10220575, train_accuracy: 0.4062, test_Accuracy: 0.4314
Epoch: [ 0] [    3/  468] time: 1.7982, train_loss: 1.95755959, train_accuracy: 0.5938, test_Accuracy: 0.4809
Epoch: [ 0] [    4/  468] time: 2.0056, train_loss: 1.94063807, train_accuracy: 0.4688, test_Accuracy: 0.5119
Epoch: [ 0] [    5/  468] time: 2.2380, train_loss: 1.86165595, train_accuracy: 0.5078, test_Accuracy: 0.5544
Epoch: [ 0] [    6/  468] time: 2.4425, train_loss: 1.71105075, train_accuracy: 0.6797, test_Accuracy: 0.5969
Epoch: [ 0] [    7/  468] time: 2.6549, train_loss: 1.62779164, train_accuracy: 0.5859, test_Accuracy: 0.6343
Epoch: [ 0] [    8/  468] time: 2.8434,

Epoch: [ 0] [   75/  468] time: 18.2781, train_loss: 0.41006184, train_accuracy: 0.8516, test_Accuracy: 0.9023
Epoch: [ 0] [   76/  468] time: 18.4706, train_loss: 0.25366783, train_accuracy: 0.9219, test_Accuracy: 0.9060
Epoch: [ 0] [   77/  468] time: 18.6631, train_loss: 0.39034373, train_accuracy: 0.8516, test_Accuracy: 0.9082
Epoch: [ 0] [   78/  468] time: 18.8376, train_loss: 0.23831762, train_accuracy: 0.9219, test_Accuracy: 0.9040
Epoch: [ 0] [   79/  468] time: 19.0152, train_loss: 0.41166735, train_accuracy: 0.8516, test_Accuracy: 0.8999
Epoch: [ 0] [   80/  468] time: 19.2007, train_loss: 0.28209713, train_accuracy: 0.9297, test_Accuracy: 0.8971
Epoch: [ 0] [   81/  468] time: 19.4640, train_loss: 0.33747217, train_accuracy: 0.9219, test_Accuracy: 0.8986
Epoch: [ 0] [   82/  468] time: 19.7243, train_loss: 0.28696907, train_accuracy: 0.9375, test_Accuracy: 0.9054
Epoch: [ 0] [   83/  468] time: 19.9826, train_loss: 0.29722124, train_accuracy: 0.9062, test_Accuracy: 0.9124
E

Epoch: [ 0] [  149/  468] time: 33.6311, train_loss: 0.27538359, train_accuracy: 0.9141, test_Accuracy: 0.9244
Epoch: [ 0] [  150/  468] time: 33.8245, train_loss: 0.24693757, train_accuracy: 0.9219, test_Accuracy: 0.9308
Epoch: [ 0] [  151/  468] time: 34.0140, train_loss: 0.19848272, train_accuracy: 0.9453, test_Accuracy: 0.9333
Epoch: [ 0] [  152/  468] time: 34.2115, train_loss: 0.11561541, train_accuracy: 0.9688, test_Accuracy: 0.9315
Epoch: [ 0] [  153/  468] time: 34.4389, train_loss: 0.31930268, train_accuracy: 0.9453, test_Accuracy: 0.9277
Epoch: [ 0] [  154/  468] time: 34.6234, train_loss: 0.32257593, train_accuracy: 0.9141, test_Accuracy: 0.9240
Epoch: [ 0] [  155/  468] time: 34.8059, train_loss: 0.40391958, train_accuracy: 0.8984, test_Accuracy: 0.9222
Epoch: [ 0] [  156/  468] time: 35.0164, train_loss: 0.23018883, train_accuracy: 0.9453, test_Accuracy: 0.9222
Epoch: [ 0] [  157/  468] time: 35.2218, train_loss: 0.15983474, train_accuracy: 0.9609, test_Accuracy: 0.9234
E

Epoch: [ 0] [  224/  468] time: 49.2304, train_loss: 0.15858305, train_accuracy: 0.9766, test_Accuracy: 0.9346
Epoch: [ 0] [  225/  468] time: 49.3930, train_loss: 0.16603613, train_accuracy: 0.9453, test_Accuracy: 0.9346
Epoch: [ 0] [  226/  468] time: 49.5625, train_loss: 0.16736704, train_accuracy: 0.9297, test_Accuracy: 0.9346
Epoch: [ 0] [  227/  468] time: 49.7918, train_loss: 0.29663378, train_accuracy: 0.9375, test_Accuracy: 0.9372
Epoch: [ 0] [  228/  468] time: 50.0153, train_loss: 0.25693256, train_accuracy: 0.9219, test_Accuracy: 0.9397
Epoch: [ 0] [  229/  468] time: 50.2367, train_loss: 0.23850469, train_accuracy: 0.9531, test_Accuracy: 0.9418
Epoch: [ 0] [  230/  468] time: 50.4850, train_loss: 0.13297257, train_accuracy: 0.9531, test_Accuracy: 0.9427
Epoch: [ 0] [  231/  468] time: 50.6835, train_loss: 0.16499759, train_accuracy: 0.9375, test_Accuracy: 0.9439
Epoch: [ 0] [  232/  468] time: 50.8680, train_loss: 0.14171183, train_accuracy: 0.9766, test_Accuracy: 0.9446
E

Epoch: [ 0] [  298/  468] time: 64.9603, train_loss: 0.14833972, train_accuracy: 0.9531, test_Accuracy: 0.9483
Epoch: [ 0] [  299/  468] time: 65.1767, train_loss: 0.16294579, train_accuracy: 0.9531, test_Accuracy: 0.9462
Epoch: [ 0] [  300/  468] time: 65.4201, train_loss: 0.16175169, train_accuracy: 0.9375, test_Accuracy: 0.9449
Epoch: [ 0] [  301/  468] time: 65.6175, train_loss: 0.23955519, train_accuracy: 0.9297, test_Accuracy: 0.9449
Epoch: [ 0] [  302/  468] time: 65.8150, train_loss: 0.11767542, train_accuracy: 0.9766, test_Accuracy: 0.9454
Epoch: [ 0] [  303/  468] time: 66.0195, train_loss: 0.17728692, train_accuracy: 0.9219, test_Accuracy: 0.9457
Epoch: [ 0] [  304/  468] time: 66.2239, train_loss: 0.08784426, train_accuracy: 0.9844, test_Accuracy: 0.9482
Epoch: [ 0] [  305/  468] time: 66.4184, train_loss: 0.15665667, train_accuracy: 0.9531, test_Accuracy: 0.9498
Epoch: [ 0] [  306/  468] time: 66.6109, train_loss: 0.17986161, train_accuracy: 0.9453, test_Accuracy: 0.9506
E

Epoch: [ 0] [  373/  468] time: 81.5559, train_loss: 0.12366275, train_accuracy: 0.9531, test_Accuracy: 0.9502
Epoch: [ 0] [  374/  468] time: 81.7394, train_loss: 0.20562696, train_accuracy: 0.9453, test_Accuracy: 0.9530
Epoch: [ 0] [  375/  468] time: 81.9060, train_loss: 0.10571376, train_accuracy: 0.9766, test_Accuracy: 0.9536
Epoch: [ 0] [  376/  468] time: 82.0985, train_loss: 0.13228717, train_accuracy: 0.9375, test_Accuracy: 0.9527
Epoch: [ 0] [  377/  468] time: 82.2820, train_loss: 0.25934079, train_accuracy: 0.9297, test_Accuracy: 0.9504
Epoch: [ 0] [  378/  468] time: 82.4585, train_loss: 0.11433642, train_accuracy: 0.9531, test_Accuracy: 0.9493
Epoch: [ 0] [  379/  468] time: 82.6360, train_loss: 0.25603846, train_accuracy: 0.9375, test_Accuracy: 0.9470
Epoch: [ 0] [  380/  468] time: 82.8554, train_loss: 0.16462746, train_accuracy: 0.9609, test_Accuracy: 0.9462
Epoch: [ 0] [  381/  468] time: 83.1506, train_loss: 0.11756428, train_accuracy: 0.9688, test_Accuracy: 0.9450
E

Epoch: [ 0] [  447/  468] time: 99.8989, train_loss: 0.12970647, train_accuracy: 0.9531, test_Accuracy: 0.9502
Epoch: [ 0] [  448/  468] time: 100.1901, train_loss: 0.10099588, train_accuracy: 0.9688, test_Accuracy: 0.9517
Epoch: [ 0] [  449/  468] time: 100.3856, train_loss: 0.11357155, train_accuracy: 0.9688, test_Accuracy: 0.9544
Epoch: [ 0] [  450/  468] time: 100.5950, train_loss: 0.11801068, train_accuracy: 0.9844, test_Accuracy: 0.9559
Epoch: [ 0] [  451/  468] time: 100.7925, train_loss: 0.10552384, train_accuracy: 0.9766, test_Accuracy: 0.9580
Epoch: [ 0] [  452/  468] time: 100.9800, train_loss: 0.19256677, train_accuracy: 0.9453, test_Accuracy: 0.9595
Epoch: [ 0] [  453/  468] time: 101.2153, train_loss: 0.14474468, train_accuracy: 0.9531, test_Accuracy: 0.9585
Epoch: [ 0] [  454/  468] time: 101.4118, train_loss: 0.15418515, train_accuracy: 0.9609, test_Accuracy: 0.9560
Epoch: [ 0] [  455/  468] time: 101.6133, train_loss: 0.24077219, train_accuracy: 0.9219, test_Accuracy: 